# Testing OCR for the Timbuktu Chronicles
This file contains scripts that test the accuracy of various OCR outputs produced for samples of Tarikh al-Fattash and Tarikh al-Soudan. Both samples will be tested against manually-corrected versions.

The code utilizes CAMeL's pre-processing tools, like de-diacritization. More information on this can be seen from CAMeL's [Guided Tour on Google Colab](https://colab.research.google.com/drive/1Y3qCbD6Gw1KEw-lixQx1rI6WlyWnrnDS?authuser=3)


The accuracy tests are produced by Character Error Rate and Word Error Rate. More information on these error rates can be found here:
- https://sites.google.com/site/textdigitisation/qualitymeasures/computingerrorrates
- https://towardsdatascience.com/evaluating-ocr-output-quality-with-character-error-rate-cer-and-word-error-rate-wer-853175297510#5aec

4 modules for these:
- https://pypi.org/project/fastwer/
- https://pypi.org/project/jiwer/
- https://pypi.org/project/asr_evaluation/
- https://pypi.org/project/asrtoolkit/




In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# Fattash Arabic only
fattashCorrect = open("/content/gdrive/My Drive/OCR_data/fattash-arabic-only-corrected.txt", "r").read()
fattashHT = open("/content/gdrive/My Drive/OCR_data/fattash-arabic-only.txt", "r").read()
fattashES = open("/content/gdrive/My Drive/OCR_data/fattash-simorgh-arabic-only.txt", "r").read()

# Soudan Arabic Only
soudanCorrect = open("/content/gdrive/My Drive/OCR_data/tarikh-as-soudan-pgs9-26-corrected.txt", "r").read()
soudanHT = open("/content/gdrive/My Drive/OCR_data/tarikh-as-soudan-ara-pgs9-26.txt", "r").read()
soudanES = open("/content/gdrive/My Drive/OCR_data/soudan-simorgh-arabic.txt", "r").read()

In [ ]:
# Pre-process CORRECT and HathiTrust via the following CAMeL functions: unicode normalizations, orthographic normalization, and de-diacritization
%pip install camel-tools
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar

     |████████████████████████████████| 61kB 2.9MB/s 
     |████████████████████████████████| 778kB 8.6MB/s 
     |████████████████████████████████| 256kB 43.7MB/s 
     |████████████████████████████████| 3.0MB 44.6MB/s 
     |████████████████████████████████| 1.2MB 22.1MB/s 
     |████████████████████████████████| 901kB 38.0MB/s 
  Created wheel for camel-tools: filename=camel_tools-1.1.0-cp37-none-any.whl size=96910 sha256=3b477fe9cd56a8e51499ee39b32dbf685a7b6f9b3c30a2c714c7c87d1ca12435
  Stored in directory: /root/.cache/pip/wheels/ff/8d/46/949198daef42f9e3bf64ed645a4af05816f47aa7677ac74589
  Created wheel for camel-kenlm: filename=camel_kenlm-2020.11.2-cp37-cp37m-linux_x86_64.whl size=2309388 sha256=4b45b4c1272783d04bcd35191202370710e632be894d364a7bec3c784f0e6bd4
  Stored in directory: /root/.cache/pip/wheels/a4/df/36/46723b5cb1a11c345ad10813f13d2ce2452e95b0aaca7ad332
Successfully built camel-tools camel-kenlm


In [ ]:
# function to pre-process Arabic text
# not sure if everything in this function from CAMeL Tools is necessary
def preprocess(text):
  # turns stuff like "ﷺ" into the full text
  text = normalize_unicode(text)

  # Normalize alef variants to 'ا' 
  text = normalize_alef_ar(text)

  # Normalize alef maksura 'ى' to yeh 'ي'
  text = normalize_alef_maksura_ar(text)

  # Normalize teh marbuta 'ة' to heh 'ه'
  text = normalize_teh_marbuta_ar(text)

  # Remove diacritization (tashkeel / harakat)
  text = dediac_ar(text)

  return text



In [ ]:
# fattash arabic only:
fattashCorrect = preprocess(fattashCorrect)
fattashHT = preprocess(fattashHT)
fattashES = preprocess(fattashES)

# soudan arabic only:
soudanCorrect = preprocess(soudanCorrect)
soudanHT = preprocess(soudanHT)
soudanES = preprocess(soudanES)


In [ ]:
soudanCorrect

'<pb n="9"/>"\n(1) بسم الله الرحمان الرحيم وصلي الله علي\nسيدنا محمد نبيه واله وصحبه وسلم\n\\\nالحمد لله المنفرد بالملك والبقاء والقدره والثناء المحيط بعلمه بجميع الاشياء\nيعلم ما كان وما يكون وان لو كان كيف يكون لا يعزب عنه مثقال ذره في الارض\nولا في السماء يؤتي الملك من يشاء وينزع الملك ممن يشاء سبحانه من ملك\nقادر وعزيز قاهر الذي قهر1 عباده بالموت والفناء وهو الاول بلا ابتداء والاخر\nبلا انتهاء والصلاه والسلام علي سيد الاولين والاخرين سيدنا ومولانا محمد\nخاتم الرسل والانبياء وعلي اله واصحابه الطيبين الطاهرين من اهل الصفوه\nوالاعتناء صلي الله عليه وعليم اجمعين وسلم 2 صلاه وسلاما بلا انقطاع3 ولا\nانقضاء و بعد\nفقد ادركنا اسلافنا المتقدمين 4 اكثر ما يتوانسون به في مجالسهم ذكر\nالصحابه والصالحين رضي الله عنهم ورحمهم ثم ذكر 5 اشياخ بلادهم وملوكها\nوسيرهم وقصصهم وانبائهم 6 وايامهم ووفياتهم وهو احلي ما يرون واشهي\nما يتذاكرون حتي انقرض ذلك الجيل ومضي رحمه الله تعالي عليهم واما الجيل\n<pb n="10"/>"\n ٢ \nالثاني ما كان فيهم من له الاعتناء بذلك ولا من يقتدي بطريق السلف الماضين\nولا من له همه 

### FastWER
Following code from https://github.com/kennethleungty/OCR-Metrics-CER-WER/blob/main/OCR_Metrics_CER_WER_Colab.ipynb

In [ ]:
!pip install pybind11
!pip install fastwer

     |████████████████████████████████| 204kB 4.3MB/s 
  Using cached https://files.pythonhosted.org/packages/cb/4d/a497cf1554862372a25576ac38c60e4ae4494cd6795aa52a21eab041cfd2/fastwer-0.1.3.tar.gz
  Created wheel for fastwer: filename=fastwer-0.1.3-cp37-cp37m-linux_x86_64.whl size=611853 sha256=4509deeb5da95b7621908b6603ece4ae0ef641cbd723a4d12877f3ca3be3b202
  Stored in directory: /root/.cache/pip/wheels/42/ec/80/4d991f6c997d6c21721c02d714967be2dd0a467d20c5f78265
Successfully built fastwer


In [ ]:
import fastwer

In [ ]:
# Obtain Sentence-Level Character Error Rate (CER)
# fastwer.score_sent(output, ref, char_level=True)
# Obtain Sentence-Level Word Error Rate (WER)
# fastwer.score_sent(output, ref)

# WORD ERROR RATE

print("Fattash Sample OCR WER:")
print("HathiTrust: " + str(fastwer.score_sent(fattashHT, fattashCorrect)))
print("eScriptorium: " + str(fastwer.score_sent(fattashES, fattashCorrect)))

print()

print("Soudan Sample OCR WER:")
print("HathiTrust: " + str(fastwer.score_sent(soudanHT, soudanCorrect)))
print("eScriptorium: " + str(fastwer.score_sent(soudanES, soudanCorrect)))

Fattash Sample OCR WER:
HathiTrust: 27.5211
eScriptorium: 26.4233

Soudan Sample OCR WER:
HathiTrust: 18.277
eScriptorium: 22.2555


In [ ]:
print("Fattash Sample OCR CER:")
print("HathiTrust: " + str(fastwer.score_sent(fattashHT, fattashCorrect, char_level=True)))
print("eScriptorium: " + str(fastwer.score_sent(fattashES, fattashCorrect, char_level=True)))

print()

print("Soudan Sample OCR CER:")
print("HathiTrust: " + str(fastwer.score_sent(soudanHT, soudanCorrect, char_level=True)))
print("eScriptorium: " + str(fastwer.score_sent(soudanES, soudanCorrect, char_level=True)))

Fattash Sample OCR CER:
HathiTrust: 6.3078
eScriptorium: 5.5518

Soudan Sample OCR CER:
HathiTrust: 5.1128
eScriptorium: 5.5535


### JIWER

Does not have CER

In [ ]:
# fastwer did not install.
%pip install jiwer

     |████████████████████████████████| 51kB 5.0MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149828 sha256=d7fae63022c3ab0f9299665f262f4256b2ab839358e3357627753ae84001a0ab
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
from jiwer import wer

In [ ]:
# wer(ground_truth, hypothesis)
print("Fattash Sample OCR WER:")
error1 = wer(fattashCorrect, fattashHT)
print("HathiTrust: " + str(error1))

error2 = wer(fattashCorrect, fattashES)
print("eScriptorium: " + str(error2))

print()
print("Soudan Sample OCR WER:")
error3 = wer(soudanCorrect, soudanHT)
print("HathiTrust: " + str(error3))
error4 = wer(soudanCorrect, soudanES)
print("eScriptorium: " + str(error4))



Fattash Sample OCR WER:
HathiTrust: 0.27388535031847133
eScriptorium: 0.25961783439490443

Soudan Sample OCR WER:
HathiTrust: 0.18258006584854833
eScriptorium: 0.2167015863513918
